In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [68]:
from ase.build import mx2
from gpaw import GPAW, PW, FermiDirac

structure = mx2(formula='MoS2', kind='2H', a=3.184, thickness=3.127,size=(1, 1, 1), vacuum=3.5)
structure.pbc = (1, 1, 0)

calc = GPAW(mode=PW(400),
            xc='PBE', 
            setups={'Mo': '6'},
            parallel={'kpt': 1},
            kpts={'size': (4,4,1),'gamma': True},
            occupations=FermiDirac(0.01))

structure.set_calculator(calc)
structure.get_potential_energy()
calc.write('MoS2_gs.gpw', 'all')


  ___ ___ ___ _ _ _  
 |   |   |_  | | | | 
 | | | | | . | | | | 
 |__ |  _|___|_____|  1.3.0
 |___|_|             

User:   Yaroslav@MacBook-Pro-Aroslav-2.local
Date:   Tue Mar 20 12:08:51 2018
Arch:   x86_64
Pid:    7137
Python: 2.7.14
gpaw:   /Users/Yaroslav/Library/Python/2.7/lib/python/site-packages/gpaw
_gpaw:  /Users/Yaroslav/Library/Python/2.7/lib/python/site-packages/_gpaw.so
ase:    /usr/local/lib/python2.7/site-packages/ase (version 3.15.0)
numpy:  /Users/Yaroslav/Library/Python/2.7/lib/python/site-packages/numpy (version 1.13.3)
scipy:  /usr/local/lib/python2.7/site-packages/scipy (version 0.19.1)
units:  Angstrom and eV
cores:  1

Input parameters:
  kpts: {gamma: True,
         size: (4, 4, 1)}
  mode: {ecut: 400.0,
         name: pw}
  occupations: {name: fermi-dirac,
                width: 0.01}
  setups: {Mo: 6}
  xc: PBE

Timing:                              incl.     excl.
-----------------------------------------------------------
Full diag:                        

In [70]:
calc.diagonalize_full_hamiltonian()
calc.write('gs_MoS2.gpw', mode='all')

Diagonalizing full Hamiltonian (1605 lowest bands)
Matrix size (min, max): 1605, 1625
Approximate memory used per core to store H_GG, S_GG: 120.878 MB
Notice: Up to twice the amount of memory might be allocated
during diagonalization algorithm.
The least memory is required when the parallelization is purely
over states (bands) and not k-points, set GPAW(..., parallel={'kpt': 1}, ...).
79.5704193115s |----------------------------------------| Time: 81.556s
Writing to gs_MoS2.gpw (mode='all')



In [71]:
from gpaw.response.g0w0 import G0W0
gw = G0W0(calc='gs_MoS2.gpw',
              bands=(8, 11),
              ecut=80,
              truncation='2D',
              nblocksmax=True,
              anisotropy_correction=True,
              filename='MoS2_g0w0_%s' %ecut,
              savepckl=True)
result=gw.calculate()

In [83]:
Ef=calc.get_fermi_level()
enk=result['qp'][0]-Ef

In [85]:
from gpaw.response import bse 

ecut = 50
eshift = 0.8

bse = bse.BSE('gs_MoS2.gpw',
          ecut=ecut,
          valence_bands=[8],
          conduction_bands=[9,10],
          truncation='2D',
          nbands=50,
          eshift=eshift,
          mode='BSE',
          integrate_gamma=1,
          txt='bse_MoS2_trun.txt')

In [86]:
from gpaw.kpt_descriptor import KPointDescriptor
from gpaw.wavefunctions.pw import PWDescriptor
from gpaw.response.chi0 import Chi0
from gpaw.response.kernels import get_coulomb_kernel
from gpaw.response.kernels import get_integrated_kernel
from gpaw.response.wstc import WignerSeitzTruncatedCoulomb
from gpaw.response.pair import PairDensity

In [87]:
nK = bse.kd.nbzkpts
nc=bse.nc
nv=bse.nv
ibz_to_bz=bse.kd.bz2ibz_k

In [224]:
H=np.zeros((nK,nK,nK,nc,nc,nv),dtype=np.complex)

In [225]:
for k1 in range(nK):
    for k2 in range(nK):
        for k3 in range(nK):
            for c1 in range(nc):
                for c2 in range(nc):
                    for v in range(nv):
                        H[k1,k2,k3,c1,c2,v]=(enk[ibz_to_bz[k1]][c1]+enk[ibz_to_bz[k2]][c2]-enk[ibz_to_bz[k3]][v])

In [226]:
H.shape

(16, 16, 16, 2, 2, 1)

In [115]:
bse.q_c=[0.0, 0.0, 0.0]
bse.wstc=None

In [ ]:
qd0 = KPointDescriptor([bse.q_c])
pd0 = PWDescriptor(bse.ecut, bse.calc.wfs.gd, complex, qd0)
v_G = get_coulomb_kernel(pd0, bse.kd.N_c, truncation=bse.truncation, wstc=None)
bse.pair = PairDensity(bse.calc, bse.ecut, world=serial_comm,txt='pair.txt')
bse.get_screened_potential()
paw_corrections = bse.pair.initialize_paw_corrections(pd0)
get_pair = bse.pair.get_kpoint_pair
get_rho = bse.pair.get_pair_density

In [243]:
bands=[8,9,10]
con_bands=[9,10]
val_bands=[8]
fcb=9
fvb=8

In [257]:
rho=np.zeros((nK,len(bands),len(bands),len(v_G)),dtype=np.complex)
occ_diff=np.zeros((nK,len(bands),len(bands)),dtype=np.complex)
bar=progressbar.ProgressBar()
for k in bar(range(nK)):
    for n,m in product(bands,bands):
        pair = get_pair(pd0, 0, k,n, n+1, m, m+1)
        occ_diff[k, n-fvb,m-fvb] = pair.get_occupation_differences(np.array([n]), np.array([m]))
        rho[k, n-fvb,m-fvb] = get_rho(pd0, pair, np.array([n]), np.array([m]),optical_limit=True,
                                              direction=0,Q_aGii=Q_aGii,extend_head=False)

100% |########################################################################|


In [245]:
from itertools import product
import progressbar
bar=progressbar.ProgressBar()
for k1 in bar(range(nK)):
    for k2 in range(nK):
        for c1 in con_bands:
            for c2 in con_bands:
                kpt1_1 = bse.pair.get_k_point(0, k1, c1, c1+1)
                kpt2_1 = bse.pair.get_k_point(0, k1, c2, c2+1)
                        
                k_2=bse.kd.find_k_plus_q(bse.qd.bzk_kc[k2], [kpt1_1.K])[0]
                kpt1_2 = bse.pair.get_k_point(0, k_2, c1, c1+1)
                kpt2_2 = bse.pair.get_k_point(0, k_2, c2, c2+1)
                        
                rho1, iq = bse.get_density_matrix(kpt1_1,kpt1_2)
                rho2, iq = bse.get_density_matrix(kpt2_1,kpt2_2)
                W1 = np.dot(np.dot(rho1.conj(),bse.W_qGG[iq]),np.swapaxes(rho2, 1, 2))
                W2 = np.dot(np.dot(rho2.conj(),bse.W_qGG[iq]),np.swapaxes(rho1, 1, 2))
                        
                H[k1,k2,:,c1-fcb,c2-fcb,:]+=(W1-W2).reshape(1)[0]                  

100% |########################################################################|


In [265]:
bar=progressbar.ProgressBar()
for k1 in bar(range(nK)):
    for k2 in range(nK):
        for c in con_bands:
            for v in val_bands:
                kpt1_1 = bse.pair.get_k_point(0, k1, c+fcb, c+fcb+1)
                kpt2_1 = bse.pair.get_k_point(0, k1, v+fvb, v+fvb+1)
                        
                k_2=bse.kd.find_k_plus_q(bse.qd.bzk_kc[k2], [k1])[0]
                kpt1_2 = bse.pair.get_k_point(0, k_2, c+fcb, c+fcb+1)
                kpt2_2 = bse.pair.get_k_point(0, k_2, v+fvb, v+fvb+1)
                        
                rho1, iq = bse.get_density_matrix(kpt1_1,kpt1_2)
                rho2, iq = bse.get_density_matrix(kpt2_1,kpt2_2)
                W = np.dot(np.dot(rho1.conj(),bse.W_qGG[iq]),np.swapaxes(rho2, 1, 2))
                V = np.dot(rho[k1,c-fvb,c-fvb].conj()*v_G,rho[k2,v-fvb,v-fvb].T)
                        
                H[k1,:,k2,c-fcb,:,v-fvb]-=(W-V).reshape(1)[0]                  

100% |########################################################################|


In [266]:
bar=progressbar.ProgressBar()
for k1 in bar(range(nK)):
    for k2 in range(nK):
        for c in con_bands:
            for v in val_bands:
                kpt1_1 = bse.pair.get_k_point(0, k1, c+fcb, c+fcb+1)
                kpt2_1 = bse.pair.get_k_point(0, k1, v+fvb, v+fvb+1)
                        
                k_2=bse.kd.find_k_plus_q(bse.qd.bzk_kc[k2], [k1])[0]
                kpt1_2 = bse.pair.get_k_point(0, k_2, c+fcb, c+fcb+1)
                kpt2_2 = bse.pair.get_k_point(0, k_2, v+fvb, v+fvb+1)
                        
                rho1, iq = bse.get_density_matrix(kpt1_1,kpt1_2)
                rho2, iq = bse.get_density_matrix(kpt2_1,kpt2_2)
                W = np.dot(np.dot(rho1.conj(),bse.W_qGG[iq]),np.swapaxes(rho2, 1, 2))
                V = np.dot(rho[k1,c-fvb,c-fvb].conj()*v_G,rho[k2,v-fvb,v-fvb].T)
                        
                H[:,k1,k2,:,c-fcb,v-fvb]-=(W-V).reshape(1)[0]                  

100% |########################################################################|
